# Use Granite Code Hosted on Replicate

## Introduction

This notebook demonstrates using inference calls against a model hosted on [Replicate](https://replicate.com/).  To see how you can use [Ollama](https://ollama.com/) to host models locally instead, see the [Continue VSCode](Continue_VSCode/Continue_VSCode.ipynb) recipe.


## Replicate Credit

To remove a barrier to entry to try the Granite Code models on the Replicate platform,
use [this link](https://replicate.com/invites/a8717bfe-2f3d-4a52-88ed-1356231cdf03) to add a
small amount of credit to your Replicate account.


## Install Replicate package

In [ ]:
!pip install replicate

## Provide your API Token

This guide will demonstrate a basic inference call using the `replicate` package.

To establish an authenticated session, provide your [Replicate API Token](https://replicate.com/account/api-tokens)
to the cell when prompted below.


In [ ]:
import getpass, os

replicate_api_token = getpass.getpass()

os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

## Choose a Model

Two Granite Code models are available in the [`ibm-granite`](https://replicate.com/ibm-granite) org at Replicate.


In [ ]:
model_id = "ibm-granite/granite-8b-code-instruct-128k"
# model_id = "ibm-granite/granite-20b-code-instruct-8k"

## Define a Prompt

In [ ]:
prompt = """
    Show me a SQL query that fetches all columns for the first 50 rows
    in a table named 'users'."""

## Perform Blocking Inference

In [ ]:
import replicate

output = replicate.run(model_id,
    input={
        "system_prompt": "You are a helpful assistant",
        "max_new_tokens": 100,
        "prompt": prompt
    }
)

print(''.join(output))

## Perform Asynchronous Inference

In [ ]:
model = replicate.models.get(model_id)

prediction = replicate.predictions.create(
  model=model,
  input={"prompt": prompt}
)

# The `create` call doesn't block, so we can do other work. Here, we'll just wait.
prediction.wait()

print(''.join(prediction.output))